In [118]:
import numpy as np
import pandas as pd
import re
import inflect
from cleantext import clean
import json
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import random
import ast
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\omara\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\omara\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\omara\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [119]:
def load_data_labels(data_path, labels_path):
    with open(data_path, 'r') as f:
        data = [ast.literal_eval(line.strip()) for line in f]
    with open(labels_path, 'r') as f:
        labels = [ast.literal_eval(line.strip()) for line in f]
    return data, labels

In [120]:
data, labels = load_data_labels('dataset/dev_data_processedf.txt','dataset/dev_order_category_labelsf.txt')
print(data[:5])
print(labels[:5])

[['i', 'want', 'to', 'order', 'two', 'medium', 'pizza', 'with', 'sausage', 'and', 'black', 'olive', 'and', 'two', 'medium', 'pizza', 'with', 'pepperoni', 'and', 'extra', 'cheese', 'and', 'three', 'large', 'pizza', 'with', 'pepperoni', 'and', 'sausage'], ['five', 'medium', 'pizza', 'with', 'tomato', 'and', 'ham'], ['i', 'need', 'to', 'order', 'one', 'large', 'vegetarian', 'pizza', 'with', 'extra', 'banana', 'pepper'], ['i', 'like', 'to', 'order', 'one', 'large', 'onion', 'and', 'pepper', 'pizza'], ['i', 'll', 'have', 'one', 'pie', 'along', 'with', 'pesto', 'and', 'ham', 'but', 'avoid', 'olive']]
[[2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0], [2, 2, 2, 2, 0, 0, 0, 0, 0, 0], [2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [121]:
preds=[]
with open('dataset/model_output.txt', 'r') as f:
        preds = [ast.literal_eval(line.strip()) for line in f]
print(preds[:5])

[[2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0], [2, 2, 2, 2, 0, 0, 0, 0, 0, 0], [2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [122]:
count = 0
i = 0
for pred , lbl in zip(preds, labels):
    if pred ==  lbl:
        count += 1
    else:
        print(i)
        print(f"Predic : {pred}")
        print(f"Actual : {lbl}")
        print('----------------------------------------------------------')
    i += 1
print(f"Accuracy : {count/len(labels)}")

69
Predic : [2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 1, 1, 1]
Actual : [2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1]
----------------------------------------------------------
107
Predic : [2, 2, 2, 0, 0, 1, 1, 1, 2, 1, 1, 1, 1, 2]
Actual : [2, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2]
----------------------------------------------------------
152
Predic : [2, 2, 2, 2, 2, 0, 0, 2, 0, 0, 0, 0, 2]
Actual : [2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 2]
----------------------------------------------------------
163
Predic : [2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 0, 2, 1, 1, 2, 1, 1, 2]
Actual : [2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1]
----------------------------------------------------------
181
Predic : [2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2]
Actual : [2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]
----------------------------------------------------------
186
Predic : [2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [123]:
stopwords=set()
stopwords.add('and')
stopwords.add('m')
stopwords.add('d')
stopwords.add('i')
stopwords.add('with')
stopwords.add('only')
stopwords.add('without')
stopwords.add('with')
stopwords.add('not')
stopwords.add('ve')
stopwords.add('like')
stopwords.add('no')
stopwords.add('s')
stopwords.add('pizza')
stopwords.add('pie')
stopwords.add('thank')
stopwords.add('too')
stopwords.add('also')
stopwords.add('the')
stopwords.add('along')
stopwords.add('ill')
stopwords.add('do')
stopwords.add('nt')
stopwords.add('on')
stopwords.add('it')
stopwords.add('them')
stopwords.add('all')
stopwords.add('in')
stopwords.add('me')
stopwords.add('how')
stopwords.add('good')
stopwords.add('you')
stopwords.add('wo')
stopwords.add('please')
stopwords.add('to')
stopwords.add('even')
stopwords.add('drink')
stopwords.add('for')
stopwords.add('well')
stopwords.add('this')
stopwords.add('my')
stopwords.add('today')
stopwords.add('allergic')

In [124]:
verbs = [
    'be', 'have', 'do', 'say', 'go', 'get', 'make', 'know', 'think', 'take',
    'see', 'come', 'want', 'use', 'find', 'give', 'tell', 'work', 'call', 'try',
    'ask', 'need', 'feel', 'become', 'leave', 'put', 'mean', 'keep', 'let', 'begin',
    'seem', 'help', 'talk', 'turn', 'start', 'show', 'hear', 'play', 'run', 'move',
    'like', 'live', 'believe', 'hold', 'bring', 'happen', 'must', 'write', 'include',
    'set', 'learn', 'change', 'lead', 'understand', 'watch', 'follow', 'stop', 'create',
    'speak', 'read', 'allow', 'spend', 'grow', 'open', 'walk', 'win', 'offer', 'remember',
    'love', 'consider', 'appear', 'buy', 'wait', 'serve', 'send', 'expect', 'build',
    'continue', 'catch', 'raise', 'kill', 'turn', 'reach', 'keep', 'write', 'read', 'move',
    'seek', 'teach', 'send', 'clean', 'learn', 'forget', 'agree', 'wait', 'return', 'borrow',
    'lend', 'fit', 'stand', 'jump', 'support', 'fly', 'draw', 'drive', 'break', 'fix',
    'hear', 'shake', 'dance', 'march', 'protect', 'risk', 'manage', 'contain', 'choose',
    'connect', 'align', 'expand', 'describe', 'analyze', 'attack', 'respond', 'observe',
    'abandon', 'achieve', 'attempt', 'blame', 'compete', 'decide', 'encourage', 'estimate',
    'fear', 'guess', 'heal', 'improve', 'justify', 'link', 'mention', 'prefer', 'prepare', 'react',
    'will', 'would' ,'shall' ,'place','can','could','order'
]


In [125]:
def is_verb(word):
    return word in verbs

In [126]:
index_masks = []
for order in data:
    index_mask = [i for i, word in enumerate(order) if word in stopwords or is_verb(word)]
    index_masks.append(index_mask)


In [127]:
def flip_elements(lst):
    result = lst[:] 
    for i in range(1, len(lst) - 1):
        if lst[i - 1] == 0 and lst[i] == 1 and lst[i + 1] == 0 :
            result[i] = 0  #! Flip 1 to 0
        elif lst[i - 1] == 1  and lst[i] == 0 and lst[i + 1] == 1:
            result[i] = 1  #! Flip 0 to 1
        # elif lst[i - 1] == 2 and lst[i] != 2 and lst[i + 1] == 2:
        #     result[i] = 2

    return result

In [128]:
def fix_number_label(lst,data):
    numbers = ['zero','one','two','three','four','five','six','seven','eight','nine','ten','eleven','twelve','thirteen','fourteen','fifteen','sixteen','seventeen','eighteen','nineteen','twenty','thirty','forty','fifty','sixty','seventy','eighty','ninety','hundred']
    for i in range(len(lst)):
        if data[i] in numbers:
            lst[i] = lst[i+1] 
    return lst

In [129]:
#! calculate exact match between predicted and true labels
correct=0
i=0
for pred , lbl in zip(preds, labels):
    mask = index_masks[i] 
    pred = fix_number_label(pred,data[i])
    # pred = [pred[idx] for idx in range(len(pred)) if idx not in mask ]
    # lbl = [lbl[idx] for idx in range(len(lbl)) if idx not in mask ]
    # data[i] = [data[i][idx] for idx in range(len(data[i])) if idx not in mask ]
    pred=flip_elements(pred)
    if pred==lbl:
        correct+=1
    else:
        print(data[i])
        print(i)
        print(pred)
        print(lbl)
        print('-------------------')
    i+=1
print(correct/len(data))

['i', 'would', 'like', 'to', 'order', 'one', 'medium', 'pizza', 'with', 'italian', 'sausage', 'one', 'small', 'pizza', 'with', 'beef', 'and', 'mushroom', 'one', 'large', 'combination', 'pizza', 'and', 'four', 'large', 'pepsi']
69
[2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 1, 1, 1]
[2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1]
-------------------
['i', 'would', 'like', 'one', 'medium', 'fanta', 'one', 'pepsi', 'and', 'six', 'large', 'diet', 'coke', 'please']
107
[2, 2, 2, 0, 0, 1, 1, 1, 2, 1, 1, 1, 1, 2]
[2, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2]
-------------------
['i', 'would', 'like', 'to', 'try', 'one', 'pepperoni', 'and', 'pineapple', 'pie', 'without', 'tuna', 'please']
152
[2, 2, 2, 2, 2, 0, 0, 2, 0, 0, 0, 0, 2]
[2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 2]
-------------------
['could', 'i', 'please', 'have', 'two', 'dr', 'pepper', 'drink', 'one', 'fanta', 'drink', 'and', 'four', 'pepsi', 'drink', 'the', 'small', 'size']
163
[